In [1]:
import pandas as pd
df = pd.read_csv("../assets/TaiwanAirBox032017.csv").set_index("time")
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V507,V508,V509,V510,V511,V512,V513,V514,V515,V516
time,,,,,,,,,,,,,,,,,,,,,
1,8.234563,34.357744,41.250916,41.194228,45.758237,40.424591,36.281097,37.844685,37.835138,44.057510,...,49.152261,55.091967,44.177585,42.569721,34.833622,38.084488,44.854870,46.933081,45.600072,52.152083
2,8.074951,34.455387,39.598679,41.088167,44.970358,39.865248,37.668302,37.439057,37.459381,43.493874,...,48.586604,54.930351,44.329101,41.317196,34.651804,38.463276,45.044264,46.254293,45.387951,51.495833
3,7.918439,34.622896,38.476926,41.116787,44.305423,39.256782,39.073064,37.170659,37.277562,43.126450,...,48.238119,54.718230,44.596777,40.512650,34.616450,38.384003,45.359416,45.830051,45.212193,51.370833
4,7.629787,35.000000,37.800159,41.273497,43.786231,38.687338,38.924916,37.083538,37.134224,42.761130,...,47.738119,54.557624,45.104353,40.027802,34.730447,38.954315,45.789719,45.850253,45.290981,51.386458
5,7.533917,35.494949,37.146960,41.534969,43.316534,38.307347,38.754882,37.050830,37.390285,42.526281,...,47.731806,54.065200,45.814310,39.700529,34.954690,39.820982,46.716991,45.994192,45.903860,51.488542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,47.250000,11.000000,1.500000,4.250000,2.142857,3.000000,3.600000,1.000000,3.571429,1.857143,...,35.125000,43.250000,35.142857,27.250000,20.857143,34.142857,40.800000,32.375000,33.500000,35.000000
741,48.333333,16.333333,2.750000,6.166667,3.500000,4.250000,4.000000,2.500000,5.000000,3.000000,...,40.000000,42.222222,32.571429,19.600000,20.800000,31.500000,38.166667,32.000000,34.500000,37.500000
742,47.666667,18.833333,1.600000,4.857143,2.500000,34.114722,3.750000,2.400000,3.000000,2.400000,...,38.000000,43.000000,32.142857,14.666667,17.666667,19.000000,35.428571,30.333333,34.222222,35.333333


In [2]:
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import SplineTransformer
import numpy as np

def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )

N_SPLINES = 5

df_hour = df.assign(hour=(np.arange(len(df)) % 24))
splines = periodic_spline_transformer(24, n_splines=N_SPLINES).fit_transform(df_hour[["hour"]])
splines_df = pd.DataFrame(
    splines,
    columns=[f"spline_{i}" for i in range(splines.shape[1])],
)
df_hour_spline = df_hour.merge(splines_df, how = "left", left_on = "hour", right_index = True).drop(columns = "hour")

r2s = []
residuals = []
for target_col in df_hour_spline.columns[df_hour_spline.columns.str.startswith("V")]:
    # get ridgecv r^2
    ridgecv = RidgeCV()
    ridgecv.fit(df_hour_spline[["spline_" + str(i) for i in range(N_SPLINES)]], df_hour_spline[target_col])
    r2s.append(ridgecv.score(df_hour_spline[["spline_" + str(i) for i in range(N_SPLINES)]], df_hour_spline[target_col]))
    # get residuals
    residuals.append(df_hour_spline[target_col] - ridgecv.predict(df_hour_spline[["spline_" + str(i) for i in range(N_SPLINES)]]))

residuals = pd.concat(residuals, axis = 1)
df = residuals
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V507,V508,V509,V510,V511,V512,V513,V514,V515,V516
time,,,,,,,,,,,,,,,,,,,,,
1,-22.358607,-3.350560,12.161376,2.903251,10.606309,7.669913,2.906294,5.778229,2.862718,5.703624,...,-14.522060,-0.928594,-15.851190,6.503137,-2.633867,-13.401032,-24.934932,-6.705772,-14.808317,-5.384072
2,-22.727265,-2.360524,10.930300,3.499912,10.388256,7.813483,4.881180,5.959663,2.980044,5.812601,...,-16.368509,-1.488981,-16.900606,6.007464,-3.328351,-13.653069,-26.097468,-8.412801,-16.047267,-6.503449
3,-23.128789,-1.434996,10.277874,4.272802,10.320302,7.812956,6.904968,6.329243,3.313160,6.117834,...,-17.676725,-2.083351,-17.563483,5.802987,-3.786619,-14.158103,-26.394719,-9.493483,-16.932782,-6.862012
4,-23.641384,-0.537603,10.026263,5.113371,10.327720,7.685863,7.315358,6.823977,3.645640,6.323286,...,-18.955504,-2.630990,-17.696152,5.473077,-3.966214,-13.828486,-26.070894,-9.885183,-17.314644,-6.944375
5,-23.883089,0.136542,9.661715,5.896332,10.216735,7.512023,7.551541,7.209963,4.277438,6.455394,...,-19.699753,-3.536537,-17.316201,4.567265,-3.866668,-13.037447,-24.977780,-10.035869,-16.981274,-6.898693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,16.800354,-29.906074,-27.710055,-34.124133,-33.743746,-32.042789,-30.145067,-31.513493,-32.054437,-37.079967,...,-16.237391,-10.229357,-14.135474,-10.253995,-12.924462,-12.247159,-12.591204,-10.873668,-16.407459,-17.482315
741,17.912608,-24.393869,-26.703857,-32.870130,-32.582703,-30.761704,-30.146775,-30.299858,-30.839440,-36.408756,...,-14.489620,-11.840276,-19.637554,-18.056166,-13.916052,-15.950872,-19.078526,-13.710951,-18.175991,-16.053604
742,17.267435,-21.397685,-27.965938,-34.437476,-33.591742,-0.573554,-30.522488,-30.488432,-32.851188,-37.122370,...,-19.327675,-11.608397,-22.560249,-22.892618,-17.869151,-29.558928,-25.612275,-17.750248,-20.905371,-19.359065


In [3]:
df_long = df.reset_index().melt(id_vars=['time'], var_name='entity', value_name='value')
df_long


,time,entity,value
0,1,V1,-22.358607
1,2,V1,-22.727265
2,3,V1,-23.128789
3,4,V1,-23.641384
4,5,V1,-23.883089
...,...,...,...
383899,740,V516,-17.482315
383900,741,V516,-16.053604
383901,742,V516,-19.359065
383902,743,V516,-31.600930


In [4]:
import os
os.environ['R_HOME'] = '/usr/lib/R/'

import rpy2
from rpy2 import robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
pandas2ri.activate()

plm = importr('plm')
r_df = pandas2ri.py2rpy(df_long)
pdata = plm.pdata_frame(r_df, index = robjects.vectors.StrVector(['time', 'entity']), drop_index = True)

tests = ["levinlin", "madwu", "Pm", "invnormal", "logit"]
p_values = {}

# Loop through each test and collect its p-value
for test in tests:
    try:
        result = plm.purtest(pdata, test=test)
        p_value = result.rx2('statistic').rx2('p.value')
        p_values[test] = p_value[0]  # Assuming the p-value is the first element
    except Exception as e:
        print(f"Error running test {test}: {e}")
        p_values[test] = None
    

p_values
    

{'levinlin': 0.0,
 'madwu': 0.00010000000000000009,
 'Pm': 1.1028117705668141e-16,
 'invnormal': 0.00010000000000000024,
 'logit': 0.0001368040622338339}

Thus we have Stationarity